In [ ]:
# 지금까지 DB에 데이터를 쌓는 것을 한 것이고
# 이번엔 쌓아놓은 DB 내 데이터를 조회하는 API를 만들어보자

# 종목코드 대신 기업명으로 조회하는 기능도 추가될 것이며
# 조회일자 형식을 다르게 입력하더라도 자동으로 양식을 보정하게 할 것이다

# 클래스 구조는 다음과 같다
'''
class MarketDB:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
    
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
    
    def get_comp_info(self):
        """COMPANY_INFO 테이블에서 읽어와서 codes에 저장"""
    
    def get_daily_price(self, code, start_date=None, end_date=None):
        """KRX 종목별 시세를 DataFrame형태로 반환"""
'''

# class file을 따로 만들 예정인 클래스 내용은 아래와 같다

import pymysql
import pandas as pd
from datetime import datetime, timedelta

class MarketDB:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host="localhost", user="root", password="@@@@@@@@", db="Investar", charset="utf8")
        self.codes = {} # self.codes = dict()와 같지만, 좀 더 간결한 표현
        self.get_comp_info() # COMPANY_INFO에서 데이터 읽어와서 codes에 저장한다

    # MariaDB와 연결을 해제하는 역할을 할 것이다.
    # mk = MarketDB()로 객체를 생성했다면, del mk로 명시적으로 객체를 삭제해야 MariaDB와 연결이 해제된다
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close()
    
    def get_comp_info(self):
        """COMPANY_INFO 테이블에서 읽어와서 codes에 저장"""
    
    # 이 함수가 핵심이며 일별로 시세를 조회하는 역할을 하게 될 것이다.
    def get_daily_price(self, code, start_date=None, end_date=None):
        """KRX 종목별 시세를 DataFrame형태로 반환"""
        
        # start_date가 없으면 1년전 값을 넣어준다 ****여기 쓰일 정규표현식 관련은 5.4.5 참고
        if start_date is None: 
            one_year_ago = datetime.today() - timedelta(days=365)
            start_date = one_year_ago.strf('%Y-%m-%d')
            print("start_date if initilalized to '{}'".format(start_date))

        #######################################################

        sql = f"""
        SELECT *
        FROM DAILY_PRICE
        WHERE CODE = '{code}'
        AND DATE >= '{start_date}'
        AND DATE <= '{end_date}'
        """

        df = pd.read_sql(sql, self.conn)
        df.index = df['date']